In [215]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [216]:
#Directory of the python scripts that need to be imported - passed to sys.path.append()
SCRIPTS_PATH = '/content/drive/My Drive/TimbreTransformer/Scripts'

In [217]:
#The working directory passed to os.chdir()
DEFAULT_PATH = '/content/drive/My Drive/TimbreTransformer'

In [218]:
import sys
sys.path.append(SCRIPTS_PATH)

In [219]:
import os
os.chdir(DEFAULT_PATH)

In [220]:
import model
import tensorflow as tf
import math

In [221]:
MODEL_PARAMS = {'name':'flute_v1','n_fft':1024, 'input_shape_y': 10, 'srate': 22050, 'custom_loss_parameter': 1e5, 'save_dir': 'Flute_Classifier'}

In [222]:
def discriminator(input_shape):

    input_ = tf.keras.Input(shape = input_shape)


    conv1 = tf.keras.layers.Conv2D(40, (6,1), padding = 'same')(input_)

    act1 = tf.keras.activations.relu(conv1)

    n1 = tf.keras.layers.BatchNormalization(axis=1)(act1)


    pool1 = tf.keras.layers.MaxPooling2D(pool_size = (10,1))(n1)


    conv2 = tf.keras.layers.Conv2D(120, (6,1), padding = 'same')(pool1)

    act2 = tf.keras.activations.relu(conv2)

    n2 = tf.keras.layers.BatchNormalization(axis=1)(act2)


    pool2 = tf.keras.layers.MaxPooling2D(pool_size = (10,1))(n2)

    pool3 = tf.keras.layers.MaxPooling2D(pool_size = (1,10))(pool2)


    flat = tf.keras.layers.Flatten()(pool3)


    dense1 = tf.keras.layers.Dense(10)(flat)

    
    act3 = tf.keras.activations.relu(dense1)


    output = tf.keras.layers.Dense(1, activation = 'sigmoid')(act3)

    
    disc = tf.keras.Model(inputs=input_, outputs=output, name="discriminator")

    
    return disc

In [223]:
def generator(input_shape):

    input_ = tf.keras.Input(shape = input_shape)


    conv1 = tf.keras.layers.Conv2D(40, (6,1), padding = 'same')(input_)

    act1 = tf.keras.activations.relu(conv1)

    n1 = tf.keras.layers.BatchNormalization(axis=1)(act1)


    pool1 = tf.keras.layers.MaxPooling2D(pool_size = ((input_shape[0]/50),1))(n1)


    up3 = tf.keras.layers.Conv2DTranspose(60, ((input_shape[0]%100),1), strides = (int((input_shape[0]-(input_shape[0]%100))/50),1), output_padding = (0,0))(pool1)

    act6 = tf.keras.activations.relu(up3)

    n6 = tf.keras.layers.BatchNormalization(axis=1)(act6)

  
    conv2 = tf.keras.layers.Conv2D(120, (3,1), padding = 'same')(n6)
    
    act2 = tf.keras.activations.relu(conv2)

    n2 = tf.keras.layers.BatchNormalization(axis=1)(act2)

    
    conv6 = tf.keras.layers.Conv2D(1, (3,1), padding = 'same', activation='sigmoid')(n2)

    out = tf.math.multiply(conv6, 1e4)

    gen = tf.keras.Model(inputs=input_, outputs=out, name="generator")
 
    return gen

In [224]:
disc = discriminator((513,10,1))

In [225]:
gen = generator((513,10,1))

In [226]:
disc.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_59 (InputLayer)        [(None, 513, 10, 1)]      0         
_________________________________________________________________
conv2d_145 (Conv2D)          (None, 513, 10, 40)       280       
_________________________________________________________________
tf.nn.relu_166 (TFOpLambda)  (None, 513, 10, 40)       0         
_________________________________________________________________
batch_normalization_145 (Bat (None, 513, 10, 40)       2052      
_________________________________________________________________
max_pooling2d_103 (MaxPoolin (None, 51, 10, 40)        0         
_________________________________________________________________
conv2d_146 (Conv2D)          (None, 51, 10, 120)       28920     
_________________________________________________________________
tf.nn.relu_167 (TFOpLambda)  (None, 51, 10, 120)     

In [227]:
gen.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_60 (InputLayer)        [(None, 513, 10, 1)]      0         
_________________________________________________________________
conv2d_147 (Conv2D)          (None, 513, 10, 40)       280       
_________________________________________________________________
tf.nn.relu_169 (TFOpLambda)  (None, 513, 10, 40)       0         
_________________________________________________________________
batch_normalization_147 (Bat (None, 513, 10, 40)       2052      
_________________________________________________________________
max_pooling2d_106 (MaxPoolin (None, 51, 10, 40)        0         
_________________________________________________________________
conv2d_transpose_31 (Conv2DT (None, 513, 10, 60)       31260     
_________________________________________________________________
tf.nn.relu_170 (TFOpLambda)  (None, 513, 10, 60)       0 

In [228]:
model.create_model(name=MODEL_PARAMS['name'], n_fft = MODEL_PARAMS['n_fft'], srate = MODEL_PARAMS['srate'], input_shape_y = MODEL_PARAMS['input_shape_y'], custom_loss_parameter = MODEL_PARAMS['custom_loss_parameter'], save_dir = MODEL_PARAMS['save_dir'], discriminator = discriminator, generator = generator)

INFO:tensorflow:Assets written to: Flute_Classifier/flute_v1/assets
